# 2 - Problema de produção (composição de liga)

In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
# Minério
minerios = 3
qtd_max = [1000, 2000, 3000]
coef_minerio = [-75.0 - 120.0, -90 - 100.0, -120.0 - 110.0] 
I = [i for i in range(3)]

# Metais
metais = 4
componentes = [[0.20, 0.10, 0.30, 0.30, 0.10], [0.10, 0.20, 0.30, 0.30, 0.10], [0.05, 0.05, 0.70, 0.20, 0.0]]
J = [j for j in range(4)]

# Ligas
ligas = 2
coef_liga = [750.0 - 150.0, 600.0 - 135.0]
K = [k for k in range(2)]

In [6]:
# Modelo
m = gp.Model("Liga")

# Variáveis de decisão
# Quantidade produzida da liga k
w = m.addVars (K, vtype = GRB.CONTINUOUS, obj = coef_liga, name = "w")

# Quantidade de minério i adquirida
x = m.addVars (I, vtype = GRB.CONTINUOUS, obj = coef_minerio, name = "x")

# Quantidade de metal j na composição da liga k
y = m.addVars (J, K, vtype = GRB.CONTINUOUS, obj = 0.0, name = "y")

# Restrições
# 1. Limite máximo de minério adquirido
m.addConstr(x[0] <= 1000, name = "lim_0")
m.addConstr(x[1] <= 2000, name = "lim_1")
m.addConstr(x[2] <= 3000, name = "lim_2")

# 2. Quantidade de metal
m.addConstr(y[0,0] + y[0,1] <= 0.2*x[0] + 0.1*x[1] + 0.05*x[2], name = "qtd_metal_0")
m.addConstr(y[1,0] + y[1,1] <= 0.1*x[0] + 0.2*x[1] + 0.05*x[2], name = "qtd_metal_1")
m.addConstr(y[2,0] + y[2,1] <= 0.3*x[0] + 0.3*x[1] + 0.7*x[2], name = "qtd_metal_2")
m.addConstr(y[3,0] + y[3,1] <= 0.3*x[0] + 0.3*x[1] + 0.2*x[2], name = "qtd_metal_3")

# 3. Liga produzida
m.addConstr(w[0] <= y[0,0] + y[1,0] + y[2,0] + y[3,0], name = "liga_0")
m.addConstr(w[1] <= y[1,0] + y[1,1] + y[2,1] + y[3,1], name = "liga_1")

# 4. Proporção
m.addConstr(y[0,0] <= 0.8*w[0] , name = "prop_0")
m.addConstr(y[1,0] <= 0.3*w[0] , name = "prop_1")
m.addConstr(y[3,0] >= 0.5*w[0] , name = "prop_2")
m.addConstr(y[1,1] >= 0.4*w[1] , name = "prop_3")
m.addConstr(y[1,1] <= 0.6*w[1] , name = "prop_4")
m.addConstr(y[2,1] >= 0.3*w[1] , name = "prop_5")
m.addConstr(y[3,1] <= 0.7*w[1] , name = "prop_6")

# Função de objetivo = Maximização
m. ModelSense = GRB.MAXIMIZE

m.update()
m.write("Modelo_Liga.lp")

In [7]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 16 rows, 13 columns and 47 nonzeros
Model fingerprint: 0x9807bdae
Coefficient statistics:
  Matrix range     [5e-02, 1e+00]
  Objective range  [2e+02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 3e+03]
Presolve removed 5 rows and 3 columns
Presolve time: 0.01s
Presolved: 11 rows, 10 columns, 45 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6072314e+06   4.724722e+02   0.000000e+00      0s
       6    1.2906250e+06   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.290625000e+06


In [8]:
print(f"Função Objetivo = {m.ObjVal}")

print()
print("Quantidade de Minério Adquirida")
for i in I:
    print(f"{x[i].VarName} = {x[i].x}")

print()
print("Quantidade de Metal na Composição das Ligas")
for j in J:
    for k in K:
        print(f"{y[j,k].VarName} = {y[j,k].x}")

print()
print("Quantidade de Liga Produzida")
for k in K:
    print(f"{w[k].VarName} = {w[k].x}")

print()
print("Checagem das Proporções")
print(f"Limite máximo para y[0,0] = {0.8*w[0].x} -> y[0,0] = {y[0,0].x}")
print(f"Limite máximo para y[1,0] = {0.3*w[0].x} -> y[1,0] = {y[1,0].x}")
print(f"Limite mínimo para y[3,0] = {0.5*w[0].x} -> y[3,0] = {y[3,0].x}")
print(f"Limite mínimo para y[1,1] = {0.4*w[1].x} -> y[1,1] = {y[1,1].x}")
print(f"Limite máximo para y[1,1] = {0.6*w[1].x} -> y[1,1] = {y[1,1].x}")
print(f"Limite mínimo para y[2,1] = {0.3*w[1].x} -> y[2,1] = {y[2,1].x}")
print(f"Limite máximo para y[3,1] = {0.7*w[1].x} -> y[3,1] = {y[3,1].x}")

Função Objetivo = 1290625.0

Quantidade de Minério Adquirida
x[0] = 1000.0
x[1] = 2000.0
x[2] = 3000.0

Quantidade de Metal na Composição das Ligas
y[0,0] = 550.0
y[0,1] = 0.0
y[1,0] = 0.0
y[1,1] = 650.0
y[2,0] = 950.0
y[2,1] = 2050.0
y[3,0] = 1500.0
y[3,1] = 0.0

Quantidade de Liga Produzida
w[0] = 3000.0
w[1] = 1625.0

Checagem das Proporções
Limite máximo para y[0,0] = 2400.0 -> y[0,0] = 550.0
Limite máximo para y[1,0] = 900.0 -> y[1,0] = 0.0
Limite mínimo para y[3,0] = 1500.0 -> y[3,0] = 1500.0
Limite mínimo para y[1,1] = 650.0 -> y[1,1] = 650.0
Limite máximo para y[1,1] = 975.0 -> y[1,1] = 650.0
Limite mínimo para y[2,1] = 487.5 -> y[2,1] = 2050.0
Limite máximo para y[3,1] = 1137.5 -> y[3,1] = 0.0
